In [1]:
from langchain_community.vectorstores import Chroma
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import UnstructuredHTMLLoader 
from langchain_openai import OpenAIEmbeddings
from langchain.chains import RetrievalQA
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.llms import HuggingFaceHub
from langchain.chains import LLMChain, ConversationChain
from langchain.prompts import PromptTemplate
from langchain.memory import ChatMessageHistory, ConversationBufferMemory,ConversationSummaryMemory

from langchain_community.llms import HuggingFaceEndpoint





In [2]:
import json

# Öffne die JSON-Datei und lade den Inhalt
with open('/Users/riccardo/Desktop/Repositorys_Github/LLM/Docs/api_token.json', 'r') as api_file:
    api_token_file = json.load(api_file)

# Extrahiere die Variable aus den Daten
api_token = api_token_file['Hugging_face_token']

In [3]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=50, 
    length_function = len)
text_splitter

In [4]:
filepath = '/Users/riccardo/Desktop/Repositorys_Github/LLM/Docs/dp177.pdf'
loader = PyPDFLoader(filepath)
chunks = loader.load_and_split(text_splitter=text_splitter)

In [6]:
for chunk in chunks:
    print("Page content: \n", chunk.page_content),
    print("Page_metadata: \n", chunk.metadata),
    print("----------------------------")

Page content: 
 ’LVNXVVLRQVSDSLHUH
’LVFXVVLRQ3DSHUV
’LVFXVVLRQ3DSHU1R
6RFLDO&XVWRP)UHH5LGHUVDQG7UDGH8QLRQ0HPEHUVKLS
LQ*HUPDQ\DQG*UHDW%ULWDLQ
E\
/DV]OR*RHUNHXQG0DUNXV3DQQHQEHUJ
%HUOLQ’HFHPEHU
’HXWVFKHV,QVWLWXWIU:LUWVFKDIWVIRUVFKXQJ%HUOLQ
.|QLJLQ/XLVH6WU%HUOLQ3KRQH )D[ ,QWHUQHWKWWSZZZGLZEHUOLQGH
Page_metadata: 
 {'source': '/Users/riccardo/Desktop/Repositorys_Github/LLM/Docs/dp177.pdf', 'page': 0}
----------------------------
Page content: 
 /G13 &RUUHVSRQGLQJDXWKRU7KHDXWKRUVZRXOGOLNHWRWKDQN%HUQG)LW]HQEHUJHU&ODXV6FKQDEHO-RKDQQHV
6FKZDU]H-RDFKLP:DJQHU$[HO:HUZDW]SDUWLFLSDQWVRIVHPLQDUVDWWKH+XPEROGW8QLYHUVLW\%HUOLQDQGWKH8QLYHUVLW\RI=ULFKDQGFRQJUHVVSDUWLFLSDQWVDW(($($/(*/0*62(3DQG9I6IRUKHOSIXOFRPPHQWVDQGVXJJHVWLRQV$OOUHPDLQLQJHUURUVDUHRXUUHVSRQVLELOLW\7KH%+36GDWDZHUHPDGHDYDLODEOHWKURXJKWKH8.’DWD$UFKLYH7KHGDWDZHUHRULJL

In [7]:
embedding_function = SentenceTransformerEmbeddings(model_name="BAAI/bge-large-zh-v1.5")

/Users/riccardo/anaconda3/envs/llm_train/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
embedding = embedding_function.embed_documents("This is a test sentence.")

print(embedding[0])
print("Dimension of Embedding: ", len(embedding[0]))

[0.030692150816321373, -0.031502220779657364, -0.017730996012687683, 0.035333432257175446, -0.028013229370117188, 0.04366171732544899, -0.017326941713690758, -0.02879144996404648, -0.018377207219600677, -0.03722976893186569, -0.006641303189098835, 0.03447132557630539, -0.022506963461637497, -0.0067087761126458645, -0.00989031232893467, -0.023188108578324318, -0.003284014528617263, 0.024761414155364037, 0.014999981969594955, -0.01783912256360054, -0.023429516702890396, 0.008491289801895618, -0.041361816227436066, 0.001655183150433004, 0.026237130165100098, -0.032837141305208206, -0.01661311462521553, 0.03826865926384926, -0.026053553447127342, -0.025660574436187744, 0.00018425437156111002, 0.057974353432655334, 0.012534989975392818, -0.026392772793769836, -0.021217787638306618, -0.004844049923121929, 0.016133053228259087, -0.007580542471259832, 0.028098614886403084, 0.0007906158571131527, 0.02106611058115959, -0.00990001205354929, 0.029389623552560806, -0.0029493358451873064, 0.00995842

In [9]:
db = Chroma.from_documents(chunks, embedding_function)

In [10]:
print("Chunks in DB:", db._collection.count())

Chunks in DB: 285


In [11]:
query = "Write a summary of the first page of the document."
retriever = db.as_retriever()
retriever.get_relevant_documents(query)

[Document(page_content='\x16\x187DEOH\x03\x14\x14\n6RFLDO\x03&XVWRP\x0f\x038QLRQ\x030HPEHUVKLS\x03DQG\x03:DJHV\n*UHDW\x03%ULWDLQ\n\x10\x03XQLRQ\x03PHPEHUVKLS\x03HTXDWLRQ\x03HPSOR\\LQJ\x03SDUWQHU\x03LQIRUPDWLRQ\x1d\x03JHQHUDO\x03UHSXWDWLRQ\x03IXQFWLRQ \x10\n/c3/c3/c3/c20/c28/c28/c20 /c3/c3/c3/c20/c28/c28/c21 /c3/c3/c3/c20/c28/c28/c22 /c3/c3/c3/c20/c28/c28/c23 /c3/c3/c3/c20/c28/c28/c24\n/c38/c82/c81/c86/c87/c68/c81/c87 /c16/c19/c17/c25/c28/c21\n/c11/c20/c19/c17/c20/c24/c24/c12/c20/c17/c20/c27/c19\n/c11/c20/c17/c23/c27/c23/c12/c16/c20/c17/c26/c26/c22\n/c11/c20/c17/c20/c20/c22/c12/c16/c20/c17/c22/c28/c22\n/c11/c20/c17/c28/c28/c20/c12/c16/c20/c17/c27/c20/c28\n/c11/c20/c17/c23/c24/c22/c12\n/c54/c82/c70/c76/c68/c79/c3/c70/c88/c86/c87/c82/c80/c3/c89/c68/c85/c76/c68/c69/c79/c72/c86\n/c51/c68/c85/c87/c81/c72/c85/c3/c83/c85/c72/c73/c17/c3/c47/c68/c69/c82/c88/c85/c3/c13/c3/c88/c81/c76/c82/c81/c3/c71/c72/c81/c86/c76/c87/c92/c3/c11/c16/c20/c12 /c19/c17/c19/c20/c19\n/c11/c19/c17/c19/c19/c23/c12/c19/c

In [12]:
llm = HuggingFaceEndpoint(repo_id='mistralai/Mistral-7B-Instruct-v0.2', 
                     huggingfacehub_api_token=api_token,  
                     model_kwargs={"max_length": 300})


Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /Users/riccardo/.cache/huggingface/token
Login successful


In [13]:
qa_with_sources = RetrievalQAWithSourcesChain.from_chain_type(
    llm = llm,
    chain_type = "stuff",  
    retriever = retriever
    )

qa_with_sources

RetrievalQAWithSourcesChain(combine_documents_chain=StuffDocumentsChain(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['question', 'summaries'], template='Given the following extracted parts of a long document and a question, create a final answer with references ("SOURCES"). \nIf you don\'t know the answer, just say that you don\'t know. Don\'t try to make up an answer.\nALWAYS return a "SOURCES" part in your answer.\n\nQUESTION: Which state/country\'s law governs the interpretation of the contract?\n=========\nContent: This Agreement is governed by English law and the parties submit to the exclusive jurisdiction of the English courts in  relation to any dispute (contractual or non-contractual) concerning this Agreement save that either party may apply to any court for an  injunction or other relief to protect its Intellectual Property Rights.\nSource: 28-pl\nContent: No Waiver. Failure or delay in exercising any right or remedy under this Agreement shall not constitute a w

In [14]:
query = "Write a summary of the key findings of the document."
qa_with_sources.invoke(query)

{'question': 'Write a summary of the key findings of the document.',
 'answer': ' The document contains encoded text and does not provide any meaningful information in its current form. It is not possible to summarize the key findings of the document without decoding it first.\n',
 'sources': ''}

In [15]:
import os 
